In [ ]:
## New notebook to carry out some of the trickier analysis on binned intensity ##
# GOAL: Would like to find a mean intensity at each box as well as a standard deviation
# Will agglomerate all points within a given range (currently 15-days and 2 degrees) and spit out an answer.

#the end vision is to have a 3-panel plot with decadal frequency change, intensity chance and f*i change
#effectively, to give a causality to changes in banded rainfall already observed in the banded rainfall plot.

In [8]:
import numpy as np
import time
import os
import netCDF4 as nc
import datetime
import matplotlib.pyplot as plt
%matplotlib notebook

In [10]:
## LOAD FROM NETCDF FILE ##
savefile = "RDA_int_hov"
ferretpath = "/Users/Siwen/Desktop/Ferret/bin/"

#netCDF output file formatting formatting
hovname = ferretpath + savefile + ".nc"
hovnc = nc.Dataset(hovname, "r")

#populate lists with intensity occupancy plots for each different time period (periods given below)
yr_beg = [1951,1951,1980,1980,1994]
yr_end = [2007,1979,2007,1993,2007]

#initialize intensity occupancy matrix storage
ints_1_hov = []
ints_2_hov = []
ints_hov = []

for yr_b, yr_e in zip(yr_beg, yr_end):

    years = str(yr_b-1900) + str("{0:0=2d}".format((yr_e-1900)%100))
    print(years)
    
    ints_1_hov.append(hovnc['int_1_hov_' + years][:])
    ints_2_hov.append(hovnc['int_2_hov_' + years][:])
    ints_hov.append(hovnc['int_hov_' + years][:])
    
hovnc.close()

5107
5179
8007
8093
9407


In [23]:
##collect function designed to aggregate all observations from 3D storage matrix - takes every ensemble (3rd dimension)
#then selections range of days and latitude range (I've mostly been using 15 day and 2 degree-latitude smoothing)

#RETURNS every non-NaN observation, as a single numpy array
#inputs: P is the full 3D matrix, dd is the day of interest(0-364), yy is the latitude bin of interest (likely 0-21)
def collect(P,dd,latbin,dayrange,latrange):
        
    ll=int((dayrange-1)/2)
    yy=int((latrange-1)/2)
    
    days = P.shape[0]
    lats = P.shape[1]
    
    Psample = P.take(range(dd-ll,dd+ll+1), mode='wrap', axis=0)
    P_collect = Psample[:,max(0,latbin-yy):min(days,latbin+yy+1),:].flatten()
    P_return = P_collect[~np.isnan(P_collect)]
    print(len(P_collect))
    print(len(P_return))
    
    return P_collect[~np.isnan(P_collect)]

In [26]:
collect(ints_hov[0], 100, 15, 15, 5)

750
24


array([ 31.55119524,  21.85476466,  16.81906197,  10.02029892,
        21.99820625,  35.89621692,  13.9070744 ,  15.32347759,
        26.63547343,  16.93527735,  25.78318451,  13.23580362,
        12.40536113,  20.09460856,  16.64219766,   9.79823179,
        13.57068573,   8.83910686,  18.16727386,  14.65620031,
        20.85166167,  24.9581063 ,  11.63875063,  17.50096139])